<a href="https://colab.research.google.com/github/Merostoroloji/VC/blob/main/FeatureEngineeringRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import  matplotlib.pyplot as plt

In [2]:
house =  pd.read_csv("house.csv", index_col = 0)
house.shape

(1460, 80)

In [3]:
house.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 80, dtype: object

In [4]:
house.MSSubClass = house.MSSubClass.astype("str")
house.dtypes

MSSubClass        object
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 80, dtype: object

In [5]:
house.isnull().sum().sum()

6965

In [6]:
missPercent = house.isnull().mean()
missPercent

MSSubClass       0.000000
MSZoning         0.000000
LotFrontage      0.177397
LotArea          0.000000
Street           0.000000
                   ...   
MoSold           0.000000
YrSold           0.000000
SaleType         0.000000
SaleCondition    0.000000
SalePrice        0.000000
Length: 80, dtype: float64

In [7]:
drop_cols = list(missPercent[missPercent > 0.1].index)
drop_cols

['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [8]:
df_drop_cols = house.drop(drop_cols, axis = 1)
df_drop_cols.shape

(1460, 74)

In [9]:
df_drop_cols.isnull().sum().sum()

609

In [10]:
df_without_na = df_drop_cols.dropna()
df_without_na.shape

(1338, 74)

In [11]:
df_without_na_dummy = pd.get_dummies(df_without_na, drop_first = True)
df_without_na_dummy.shape

(1338, 240)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = df_without_na_dummy.drop("SalePrice", axis = 1)
y = df_without_na["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

mdl = LinearRegression()
scores = cross_val_score(mdl, X_train, y_train, cv = 10)

mdl.fit(X_train, y_train)
ypred = mdl.predict(X_test)
r2 = r2_score(y_test, ypred)

result = pd.DataFrame([], columns = ["r2_mean", "r2_std", "r2_test"])
result.loc["Without NA"] = [scores.mean(), scores.std(), r2]
result

,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075


In [13]:
miss_percent = df_drop_cols.isnull().mean()
miss_percent[miss_percent > 0]

MasVnrType      0.005479
MasVnrArea      0.005479
BsmtQual        0.025342
BsmtCond        0.025342
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtFinType2    0.026027
Electrical      0.000685
GarageType      0.055479
GarageYrBlt     0.055479
GarageFinish    0.055479
GarageQual      0.055479
GarageCond      0.055479
dtype: float64

In [14]:
df_drop_cols.MasVnrType = df_drop_cols.MasVnrType.fillna("None")
df_drop_cols.MasVnrArea = df_drop_cols.MasVnrArea.fillna(0)
df_drop_cols.BsmtQual = df_drop_cols.BsmtQual.fillna("None")
df_drop_cols.BsmtCond = df_drop_cols.BsmtCond.fillna("None")
df_drop_cols.BsmtExposure = df_drop_cols.BsmtExposure.fillna("None")
df_drop_cols.BsmtFinType1 = df_drop_cols.BsmtFinType1.fillna("None")
df_drop_cols.BsmtFinType2 = df_drop_cols.BsmtFinType2.fillna("None")
df_drop_cols.Electrical = df_drop_cols.Electrical.fillna("None")
df_drop_cols.GarageType = df_drop_cols.GarageType.fillna("None")
df_drop_cols.GarageYrBlt = df_drop_cols.GarageYrBlt.fillna(0)
df_drop_cols.GarageFinish = df_drop_cols.GarageFinish.fillna("None")
df_drop_cols.GarageQual = df_drop_cols.GarageQual.fillna("None")
df_drop_cols.GarageCond = df_drop_cols.GarageCond.fillna("None")

In [15]:
df_drop_cols_dummy = pd.get_dummies(df_drop_cols, drop_first = True)
print(df_drop_cols_dummy.shape)

X = df_drop_cols_dummy.drop("SalePrice", axis = 1)
y = df_drop_cols_dummy["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

mdl = LinearRegression()
scores = cross_val_score(mdl, X_train, y_train, cv = 10)

mdl.fit(X_train, y_train)
ypred = mdl.predict(X_test)
r2 = r2_score(y_test, ypred)

result.loc["Fill NA"] = [scores.mean(), scores.std(), r2]
result

(1460, 255)


,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075
Fill NA,0.588745,0.419368,0.643133


In [16]:
df_drop_cols["LotFrontage"] = house.LotFrontage.fillna(0)
df_drop_cols["Alley"] = house.Alley.fillna("None")
df_drop_cols["FireplaceQu"] = house.FireplaceQu.fillna("None")
df_drop_cols["PoolQC"] = house.PoolQC.fillna("None")
df_drop_cols["Fence"] = house.Fence.fillna("None")
df_drop_cols["MiscFeature"] = house.MiscFeature.fillna("None")
print(df_drop_cols.shape)

(1460, 80)


In [17]:
df_drop_cols_dummy = pd.get_dummies(df_drop_cols, drop_first = True)
print(df_drop_cols_dummy.shape)

X = df_drop_cols_dummy.drop("SalePrice", axis = 1)
y = df_drop_cols_dummy["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

mdl = LinearRegression()
scores = cross_val_score(mdl, X_train, y_train, cv = 10)

mdl.fit(X_train, y_train)
ypred = mdl.predict(X_test)
r2 = r2_score(y_test, ypred)

result.loc["Full Data"] = [scores.mean(), scores.std(), r2]
result

(1460, 274)


,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075
Fill NA,0.588745,0.419368,0.643133
Full Data,0.123984,0.756375,-0.115897


In [18]:
df_new = df_drop_cols.copy()
df_new.MSSubClass.unique()

array(['60', '20', '70', '50', '190', '45', '90', '120', '30', '85', '80',
       '160', '75', '180', '40'], dtype=object)

In [19]:
df_new.MSSubClass.value_counts()/len(df_new)

20     0.367123
60     0.204795
50     0.098630
120    0.059589
30     0.047260
160    0.043151
70     0.041096
80     0.039726
90     0.035616
190    0.020548
85     0.013699
75     0.010959
45     0.008219
180    0.006849
40     0.002740
Name: MSSubClass, dtype: float64

In [20]:
category_percent = df_new.MSSubClass.value_counts()/len(df_new)
category = list(category_percent[category_percent < 0.01].index)
category

['45', '180', '40']

In [21]:
df_new.MSSubClass = df_new.MSSubClass.replace(category, "Other")
df_new.MSSubClass.value_counts()/len(df_new)

20       0.367123
60       0.204795
50       0.098630
120      0.059589
30       0.047260
160      0.043151
70       0.041096
80       0.039726
90       0.035616
190      0.020548
Other    0.017808
85       0.013699
75       0.010959
Name: MSSubClass, dtype: float64

In [22]:
df_new_dummy = pd.get_dummies(df_new, drop_first = True)
print(df_new_dummy.shape)

X = df_new_dummy.drop("SalePrice", axis = 1)
y = df_new_dummy["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

mdl = LinearRegression()
scores = cross_val_score(mdl, X_train, y_train, cv = 10)

mdl.fit(X_train, y_train)
ypred = mdl.predict(X_test)
r2 = r2_score(y_test, ypred)

result.loc["MSSubClass Simple"] = [scores.mean(), scores.std(), r2]
result

(1460, 272)


,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075
Fill NA,0.588745,0.419368,0.643133
Full Data,0.123984,0.756375,-0.115897
MSSubClass Simple,0.129578,0.752395,-0.110499


In [25]:
category_cols = [col for col in df_new.columns if df_new[col].dtype == "O"]
category_cols

['MSSubClass',
 'MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition',
 'Alley',
 'FireplaceQu',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [26]:
for col in category_cols:
    category_percent = df_new[col].value_counts()/len(df_new)
    category = list(category_percent[category_percent < 0.01].index)
    if len(category) > 1:
        df_new[col] = df_new[col].replace(category, "Other")

In [27]:
df_new_dummy = pd.get_dummies(df_new, drop_first = True)
print(df_new_dummy.shape)

X = df_new_dummy.drop("SalePrice", axis = 1)
y = df_new_dummy["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

mdl = LinearRegression()
scores = cross_val_score(mdl, X_train, y_train, cv = 10)

mdl.fit(X_train, y_train)
ypred = mdl.predict(X_test)
r2 = r2_score(y_test, ypred)

result.loc["All Category Simple"] = [scores.mean(), scores.std(), r2]
result

(1460, 217)


,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075
Fill NA,0.588745,0.419368,0.643133
Full Data,0.123984,0.756375,-0.115897
MSSubClass Simple,0.129578,0.752395,-0.110499
All Cat Simple,0.129578,0.752395,-0.110499
All Category Simple,0.750350,0.177566,0.869965


In [28]:
df_new.ExterQual.unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [29]:
df_new.ExterQual = df_new.ExterQual.replace({"Fa":1, "TA":2, "Gd":3, "Ex":4})
df_new.ExterQual.unique()

array([3, 2, 4, 1])

In [30]:
df_new_dummy = pd.get_dummies(df_new, drop_first = True)
print(df_new_dummy.shape)

X = df_new_dummy.drop("SalePrice", axis = 1)
y = df_new_dummy["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

mdl = LinearRegression()
scores = cross_val_score(mdl, X_train, y_train, cv = 10)

mdl.fit(X_train, y_train)
ypred = mdl.predict(X_test)
r2 = r2_score(y_test, ypred)

result.loc["ExterQual Ordinal"] = [scores.mean(), scores.std(), r2]
result

(1460, 215)


,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075
Fill NA,0.588745,0.419368,0.643133
Full Data,0.123984,0.756375,-0.115897
MSSubClass Simple,0.129578,0.752395,-0.110499
All Cat Simple,0.129578,0.752395,-0.110499
All Category Simple,0.750350,0.177566,0.869965
ExterQual Ordinal,0.752284,0.173773,0.868884


In [31]:
mdl.fit(X_train, y_train)
ypred = mdl.predict(X_train)
r2 = r2_score(y_train, ypred)
r2

0.9012732341290391

In [33]:
from sklearn.feature_selection import VarianceThreshold

In [36]:
selection = VarianceThreshold(0.01)
selection.fit(X_train)
X_train_new = selection.transform(X_train)
X_test_new = selection.transform(X_test)

print(X_train_new.shape)
scores = cross_val_score(mdl, X_train, y_train, cv = 10)

mdl.fit(X_train, y_train)
ypred = mdl.predict(X_test)
r2 = r2_score(y_test, ypred)

result.loc["VarianceTrhreshold "] = [scores.mean(), scores.std(), r2]
result

(1168, 196)


,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075
Fill NA,0.588745,0.419368,0.643133
Full Data,0.123984,0.756375,-0.115897
MSSubClass Simple,0.129578,0.752395,-0.110499
All Cat Simple,0.129578,0.752395,-0.110499
All Category Simple,0.750350,0.177566,0.869965
ExterQual Ordinal,0.752284,0.173773,0.868884
VarianceTrhreshold,0.752284,0.173773,0.868884


In [38]:
from sklearn.feature_selection import SelectKBest, f_regression

In [40]:
selector = SelectKBest(f_regression, k = "all")
selector.fit(X_train, y_train)
f_scores = pd.Series(selector.scores_,index =X_train.columns)
f_scores = f_scores.sort_values(ascending=False);f_scores

OverallQual             1879.151818
GrLivArea               1093.390361
ExterQual                927.122414
GarageCars               813.186816
GarageArea               744.064301
                           ...     
Condition2_Other           0.047660
BsmtFinSF2                 0.038298
RoofStyle_Other            0.010914
Neighborhood_SawyerW       0.007409
Foundation_Other           0.000051
Length: 214, dtype: float64

In [42]:
k = 100
best_cols = f_scores[:k].index
X_train_new = X_train[best_cols]
X_test_new = X_test[best_cols]
scores = cross_val_score(mdl, X_train_new, y_train, cv = 10)

mdl.fit(X_train_new, y_train)
ypred = mdl.predict(X_test_new)
r2 = r2_score(y_test, ypred)

result.loc["Selec KBest "] = [scores.mean(), scores.std(), r2]
result

,r2_mean,r2_std,r2_test
Without NA,0.547436,0.574908,0.380075
Fill NA,0.588745,0.419368,0.643133
Full Data,0.123984,0.756375,-0.115897
MSSubClass Simple,0.129578,0.752395,-0.110499
All Cat Simple,0.129578,0.752395,-0.110499
All Category Simple,0.750350,0.177566,0.869965
ExterQual Ordinal,0.752284,0.173773,0.868884
VarianceTrhreshold,0.752284,0.173773,0.868884
Selec KBest,0.806916,0.123352,0.871581


In [55]:
best_r2 = -np.inf

for k in range(2, X_train_new.shape[1]):
    best_cols = f_scores[:k].index
    X_train_new = X_train[best_cols]
    scores = cross_val_score(mdl, X_train_new, y_train, cv = 10)
    if scores.mean() > best_r2:
        best_r2 = scores.mean()
        best_k = k

print(best_k)

93


In [54]:
max(a[1])

0.871580657528707